In [140]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

In [6]:
data = pd.read_csv(r"D:\Dataset\medical_data.csv")
data.head()

,Patient_Problem,Disease,Prescription
0,"Constant fatigue and muscle weakness, struggli...",Chronic Fatigue Syndrome,"Cognitive behavioral therapy, graded exercise ..."
1,"Frequent severe migraines, sensitivity to ligh...",Migraine with Aura,"Prescription triptans, avoid triggers like bri..."
2,"Sudden weight gain and feeling cold, especiall...",Hypothyroidism,Levothyroxine to regulate thyroid hormone levels.
3,"High fever, sore throat, and swollen lymph nod...",Mononucleosis,"Rest and hydration, ibuprofen for pain."
4,"Excessive thirst and frequent urination, dry m...",Diabetes Mellitus,Insulin therapy and lifestyle changes.


In [74]:
data["Patient_Problem"][0]

'constant fatigu muscl weak struggl stay awak'

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407 entries, 0 to 406
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Patient_Problem  407 non-null    object
 1   Disease          407 non-null    object
 2   Prescription     407 non-null    object
dtypes: object(3)
memory usage: 9.7+ KB


In [86]:
x = data["Patient_Problem"]
y = data["Disease"]

In [9]:
encode = LabelEncoder()
y_encode = encode.fit_transform(y)

In [137]:
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [11]:
port = PorterStemmer()

In [21]:
x

0      Constant fatigue and muscle weakness, struggli...
1      Frequent severe migraines, sensitivity to ligh...
2      Sudden weight gain and feeling cold, especiall...
3      High fever, sore throat, and swollen lymph nod...
4      Excessive thirst and frequent urination, dry m...
                             ...                        
402    Noticeable thinning of the hair on the top of ...
403    Greenish discharge from the eyes, accompanied ...
404    Experiencing confusion, difficulty speaking, a...
405    Constant feeling of fullness in the ears and h...
406    Blood in stool, along with a change in bowel m...
Name: Patient_Problem, Length: 407, dtype: object

In [28]:
def preprocess(text):
    text = re.sub("[^a-zA-Z]"," ",text)
    text = text.lower()
    text = text.split()
    stemmed = [port.stem(i) for i in text if not i in stopwords.words("english")]
    final = " ".join(stemmed)
    return final

In [78]:
t = "Constant fatigue and muscle weakness, struggling to stay awake."
preprocess(t)

'constant fatigu muscl weak struggl stay awak'

In [54]:
data["Patient_Problem"] =data["Patient_Problem"].apply(preprocess)

In [87]:
count = CountVectorizer()
x = count.fit_transform(x)

In [91]:
print(count.get_feature_names_out())
print(len(count.get_feature_names_out()))

['abdomen' 'abdomin' 'accompani' 'achil' 'acid' 'acn' 'across' 'activ'
 'acut' 'affect' 'air' 'allergi' 'almost' 'along' 'alter' 'amount' 'ankl'
 'anxieti' 'anxiou' 'anymor' 'appar' 'appear' 'appetit' 'area' 'arm'
 'around' 'asleep' 'attack' 'avoid' 'awak' 'away' 'back' 'balanc' 'bald'
 'basi' 'becom' 'bee' 'bend' 'big' 'bite' 'bleed' 'blister' 'bloat'
 'blood' 'bloodi' 'bluish' 'blur' 'blurri' 'bodi' 'bout' 'bowel' 'breath'
 'brittl' 'brownish' 'bruis' 'burn' 'burrow' 'caus' 'certain' 'chang'
 'cheek' 'chest' 'chill' 'chronic' 'clear' 'click' 'climb' 'close'
 'cloudi' 'cold' 'color' 'come' 'common' 'concentr' 'concern' 'confus'
 'consist' 'constant' 'constantli' 'consum' 'continu' 'cool' 'coordin'
 'cough' 'counter' 'cover' 'crack' 'cramp' 'cut' 'daili' 'dark' 'day'
 'decreas' 'deep' 'deepli' 'depress' 'develop' 'diarrhea' 'diet'
 'difficult' 'difficulti' 'discharg' 'discomfort' 'dizzi' 'doubl' 'dri'
 'drink' 'ear' 'easi' 'easili' 'eat' 'end' 'energi' 'engag' 'enjoy'
 'environ' 'episo

In [84]:
x.shape

(407, 464)

In [103]:
encode = LabelEncoder()
y = encode.fit_transform(y)

In [104]:
encode.classes_

array(['Achilles Tendinitis', 'Acid Reflux',
       'Age-related Macular Degeneration', 'Allergic Rhinitis',
       'Alopecia Areata', "Alzheimer's Disease", 'Anaphylaxis',
       'Androgenetic Alopecia', 'Angina', 'Angina Pectoris', 'Anorexia',
       'Anxiety Disorder', 'Aphthous Ulcers', 'Appendicitis',
       'Arrhythmia', 'Asthma', "Athlete's Foot", 'Atopic Dermatitis',
       'Atrial Fibrillation', 'Attention Deficit Hyperactivity Disorder',
       'Basal Cell Carcinoma', "Bell's Palsy",
       'Benign Positional Vertigo', 'Bipolar Disorder',
       'Bronchial Asthma', 'Bronchitis', 'COVID-19',
       'Carpal Tunnel Syndrome', 'Cellulitis', 'Chronic Asthma',
       'Chronic Bronchitis', 'Chronic Fatigue Syndrome',
       'Chronic Migraine', 'Chronic Obstructive Pulmonary Disease',
       'Chronic Obstructive Pulmonary Disease (COPD)',
       'Cluster Headaches', 'Colorectal Cancer', 'Conjunctivitis',
       'Constipation', 'Contact Dermatitis', "Cushing's Syndrome",
       'Cyano

In [48]:
print(x.shape)
print(y.shape)

(407, 464)
(407,)


In [43]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

In [46]:
print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

(325, 464) (82, 464)
(325,) (82,)


In [142]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
# from sklearn.metrics import mean_squared_error
model = RandomForestClassifier(class_weight='balanced')
model.fit(x_train,y_train)
pred = model.predict(x_train)
pred2 = model.predict(x_test)
train_acc = accuracy_score(y_train,pred)
test_acc = accuracy_score(y_test,pred2)

In [143]:
print(train_acc)
print(test_acc)

0.9815384615384616
0.3902439024390244


In [144]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
}
grid = GridSearchCV(model,param_grid,cv=5)
grid.fit(x_train,y_train)
best_grid = grid.best_estimator_
best_grid.score(x_test,y_test)

0.4024390243902439

In [123]:
# r_model = RandomForestClassifier()
# r_model.fit(x_train,y_train)
# pre_test = r_model.predict(x_train)
# print(accuracy_score(pre_test,y_train))

0.9815384615384616

In [131]:
def make(input_data):
    pre = preprocess(input_data)
    trans = count.transform([pre])
    # data =np.array(trans).reshape(1,-1)
    result = model.predict(trans)
    return encode.inverse_transform(result)
input_text = "Persistent cough and wheezing, especially at night, with difficulty breathing."
disease = make(input_text)
print(disease)

['Bronchial Asthma']
